In [1]:
# As single_peakon, but trained on multipeakons!

In [2]:
%run base/base_eq.ipynb
%run base/base_plot.ipynb
%run base/base_ml.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [4]:
def gen_one_peakon_on_grid():
    """Returns a (feature, label) pair, where the features are the values of
    a single-peakon solution on a coarse grid, and the labels are the values of
    the single-peakon solution on a fine grid.
    """
    point, peakon = gen_one_peakon()
    return sol_on_grid(point, peakon)

def gen_two_peakon_on_grid():
    """Returns a (feature, label) pair, where the features are the values of
    a two-peakon solution on a coarse grid, and the labels are the values of
    the two-peakon solution on a fine grid.
    """
    point, twopeakon = gen_two_peakon()
    return sol_on_grid(point, twopeakon)

def gen_one_data():
    return tools.random_function(gen_one_peakon_on_grid, gen_two_peakon_on_grid)

In [ ]:
### DNN hyperparameters
hidden_units = [600] * 3
# i.e. the number of fine grid points:
logits = (fine_grid_fineness.t + 1) * (fine_grid_fineness.x + 1)
processor = IdentityProcessor()
activation = tf.nn.crelu

### Training hyperparameters #1
drop_rate = 0.0
drop_type = 'dropout'
gradient_clip = 1.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir_base = './saved_models/grid_multi_peakon_dnn_hyperparameter_testing/'
uuid = None
model_dir = model_dir_str(model_dir_base, hidden_units, logits,
                          processor=processor, activation=activation, 
                          uuid=uuid)

dnn_creator = DNNFactory(hidden_units, logits, drop_rate=drop_rate,
                         drop_type=drop_type, model_dir=model_dir, 
                         log_steps=log_steps, activation=activation, 
                         gradient_clip=gradient_clip, batch_norm=False,
                         processor=processor)

max_steps = 0
if TRAIN:
    ### Training hyperparameters #2
    batch_size = 500
    max_steps += 5000
    train_input_fn = BatchData.from_func(gen_one_data=gen_one_data, batch_size=batch_size)
    dnn = dnn_creator()
    dnn.train(input_fn=train_input_fn, max_steps=max_steps)

    batch_size = 1000
    max_steps += 5000
    train_input_fn = BatchData.from_func(gen_one_data=gen_one_data, batch_size=batch_size)
    dnn = dnn_creator()
    dnn.train(input_fn=train_input_fn, max_steps=max_steps)

    batch_size = 3000
    max_steps += 10000
    train_input_fn = BatchData.from_func(gen_one_data=gen_one_data, batch_size=batch_size)
    dnn = dnn_creator()
    dnn.train(input_fn=train_input_fn, max_steps=max_steps)